<h1>Sommaire<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Architecture-MLOps" data-toc-modified-id="Architecture-MLOps-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Architecture MLOps</a></span></li><li><span><a href="#Le-jeu-de-données" data-toc-modified-id="Le-jeu-de-données-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Le jeu de données</a></span></li></ul></li><li><span><a href="#Chargement-des-ressources" data-toc-modified-id="Chargement-des-ressources-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Chargement des ressources</a></span><ul class="toc-item"><li><span><a href="#Chargement-du-workspace" data-toc-modified-id="Chargement-du-workspace-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Chargement du workspace</a></span></li><li><span><a href="#Chargement-du-magasin-des-données" data-toc-modified-id="Chargement-du-magasin-des-données-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Chargement du magasin des données</a></span></li><li><span><a href="#Chargement-des-paramètres-de-LUIS" data-toc-modified-id="Chargement-des-paramètres-de-LUIS-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Chargement des paramètres de LUIS</a></span></li></ul></li><li><span><a href="#Exploration-et-analyse-des-données" data-toc-modified-id="Exploration-et-analyse-des-données-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exploration et analyse des données</a></span><ul class="toc-item"><li><span><a href="#Chargement-du-fichier" data-toc-modified-id="Chargement-du-fichier-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Chargement du fichier</a></span></li><li><span><a href="#Analyse-des-textes" data-toc-modified-id="Analyse-des-textes-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Analyse des textes</a></span><ul class="toc-item"><li><span><a href="#Indicateurs-statistiques" data-toc-modified-id="Indicateurs-statistiques-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Indicateurs statistiques</a></span></li></ul></li><li><span><a href="#Analyse-des-labels" data-toc-modified-id="Analyse-des-labels-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Analyse des labels</a></span><ul class="toc-item"><li><span><a href="#Distribution-des-labels" data-toc-modified-id="Distribution-des-labels-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Distribution des labels</a></span></li></ul></li><li><span><a href="#Transformation-des-données" data-toc-modified-id="Transformation-des-données-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Transformation des données</a></span></li><li><span><a href="#Split-des-données" data-toc-modified-id="Split-des-données-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Split des données</a></span></li><li><span><a href="#Enregistrement-des-datasets" data-toc-modified-id="Enregistrement-des-datasets-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Enregistrement des datasets</a></span></li></ul></li><li><span><a href="#Enregistrement-des-paramètres-de-LUIS-sur-Github" data-toc-modified-id="Enregistrement-des-paramètres-de-LUIS-sur-Github-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Enregistrement des paramètres de LUIS sur Github</a></span><ul class="toc-item"><li><span><a href="#Mise-à-jour-des-paramètres-de-LUIS" data-toc-modified-id="Mise-à-jour-des-paramètres-de-LUIS-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Mise à jour des paramètres de LUIS</a></span></li><li><span><a href="#Evaluation-du-nouveau-modèle" data-toc-modified-id="Evaluation-du-nouveau-modèle-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Evaluation du nouveau modèle</a></span></li><li><span><a href="#Enregistrement-du-Github" data-toc-modified-id="Enregistrement-du-Github-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Enregistrement du Github</a></span></li></ul></li></ul></div>

In [142]:
import urllib, zipfile

%load_ext autoreload
%autoreload 2

from notebook import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [213]:
# # Copie les notebooks et supprime les sorties.
# # Décommenter et exécuter cette cellule avant de commiter/pusher
# # les modifications du notebook sur Github.
# copy_and_clean_notebooks()

# Introduction

Ce projet a pour but la réalisation d'un MVP d'un chatbot pour aider les utilisateurs à choisir une offre de voyage.

Pour ce faire, nous allons utiliser le service LUIS de Microsoft Azure. Il permet de créer un modèle de language understanding qui va :
- Détecter l’intention générale d’une phrase (intent).
- En extraire les informations détaillées (entities).

Dans le cadre de cette première version du MVP, nous allons uniquement détecter les demandes de réservation de vol ainsi que les informations associées :
- Ville de départ.
- Ville de destination.
- Date aller souhaitée du vol.
- Date retour souhaitée du vol.
- Budget maximum pour le prix total des billets.

## Architecture MLOps

Le MLOps est une culture et une pratique d'ingénierie ML qui vise à unifier le développement de systèmes ML et leur mise en opération (Ops). Appliquer le MLOps signifie que l'on vise l'automatisation et la surveillance de toutes les étapes de la construction d'un système de ML, y compris l'intégration, les tests, la publication, le déploiement et la gestion de l'infrastructure ([en savoir plus](https://cloud.google.com/architecture/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning)).

Nous avons donc mis en place une architecture MLOps afin de pouvoir itérer rapidement sur les versions de notre MVP.

<img src="./data/img/archi_mlops.png" alt="Architecture MLOps" width="900"/>
<p style="text-align: center; text-decoration: underline;">Architecture MLOps</p>

Voici la description du code couleur :
- Les blocks en gris sont ceux qui n'ont pas encore été implémentés.
- Les blocks en bleu sont ceux qui ont déjà été implémentés.
- Les blocks en vert sont ceux qui utilisent Github.
- Le block en orange correspond au jeu de données de Microsoft.

## Le jeu de données

Nous allons utiliser un de données fournit gratuitement par Microsoft ([en savoir plus](https://www.microsoft.com/en-us/research/project/frames-dataset/download/)).

Il est composé d'un fichier JSON contenant 1369 dialogues entre 2 humains :
- Le 1er joue le rôle de l’utilisateur et le 2ème se fait passer pour le bot.
- L’utilisateur veut booker un vol et/ou un hôtel.
- Le bot va récupérer les paramètres de recherche et proposer une offre.
- L’utilisateur va donner une note représentant sa satisfaction.
- Effectué par 12 participants sur 20 jours.

Les entités dans les textes des utilisateurs sont déjà labellisées :
- Ville de départ.
- Ville d’arrivée.
- Heure de départ.
- Heure d’arrivée.
- Budget.
- Etc…

Commençons par récupérer le jeu de données :

In [3]:
# Lien vers le dataset
DATASET_URL = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+10%C2%A0-+D%C3%A9veloppez+un+chatbot+pour+r%C3%A9server+des+vacances/frames.zip"
DATASET_FN = "frames.json"

# On vérifie si le fichier est bien présent
if DATASET_FN in os.listdir(JSON_PATH):
    print("Tous les fichiers sont bien présents.")
# Sinon on télécharge et on extrait le fichier
else:
    print("Téléchargement des données en cours...")

    # On télécharge le .zip dans un fichier temporaire et on extrait les données
    tmp, _ = urllib.request.urlretrieve(DATASET_URL)
    with zipfile.ZipFile(tmp, "r") as f:
        # On extrait le fichier json
        f.extract(DATASET_FN, JSON_PATH)

    # On supprime le fichier temporaire
    urllib.request.urlcleanup()
    
    print("Téléchargement des données terminé.")

Téléchargement des données en cours...
Téléchargement des données terminé.


# Chargement des ressources

Nous allons charger toutes les ressources Azure qui vont nous permettre de créer et d'enregistrer des jeux de données.

Nous allons aussi charger les paramètres du modèle LUIS actuel.

## Chargement du workspace

In [190]:
# On charge l’espace de travail Azure Machine Learning existant
ws = Workspace.from_config()

## Chargement du magasin des données

In [191]:
# On charge le magasin de données par défaut
datastore = ws.get_default_datastore()

## Chargement des paramètres de LUIS

In [85]:
# On charge les variables d'environnement de LUIS
env = LUISEnv("../P10_02_luis/.env")

In [105]:
# On charge les paramètres du modèle
with open("../P10_02_luis/params.json") as f:
    params = json.load(f)

# Exploration et analyse des données

Cette partie représente le block Data analysis de notre architecture.

## Chargement du fichier

Le jeu de données n'est constitué que d'un seul fichier JSON. Il s'agit d'une liste de dialogues appellées `frames`.

In [14]:
# On charge les frames
with open("data/json/frames.json") as f:
    frames = json.load(f)
    
print(f"Le jeu de donnée contient {len(frames)} frames.")

Le jeu de donnée contient 1369 frames.


Voyons à quoi ressemble le premier frame :

In [6]:
frames[0]

{'user_id': 'U22HTHYNP',
 'turns': [{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
   'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
      'name': 'inform'},
     {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
       {'val': 'Caprica', 'key': 'or_city'},
       {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
       {'val': '8', 'key': 'n_adults'},
       {'val': '1700', 'key': 'budget'}],
      'name': 'inform'}],
    'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
      'name': 'inform'},
     {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
       {'val': 'Caprica', 'key': 'or_city'},
       {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
       {'val': '8', 'key': 'n_adults'},
       {'val': '1700', 'key': 'budget'}],
      'name': 'inform'}],
    'active_frame': 1,
    'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
 

## Analyse des textes

Comme indiqué dans la documentation du jeu de données ([en savoir plus](https://www.microsoft.com/en-us/research/project/frames-dataset/download/)), on retrouvera dans l'élément `turns` la liste des textes labellisés.

Voyons à quoi ressemble le premier dialogue :

In [9]:
# On affiche le premier dialogue
for turn in frames[0]["turns"]:
    print("-" * 80)
    print(turn.get("author", ""), ":", turn.get("text", ""))

--------------------------------------------------------------------------------
user : I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.
--------------------------------------------------------------------------------
wizard : Hi...I checked a few options for you, and unfortunately, we do not currently have any trips that meet this criteria.  Would you like to book an alternate travel option?
--------------------------------------------------------------------------------
user : Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.
--------------------------------------------------------------------------------
wizard : I checked the availability for this date and there were no trips available.  Would you like to select some alternate dates?
--------------------------------------------------------------------------------
user : I have no flexibilit

On s'apperçoit que le premier texte de l'utilisateur contient la majorité des informations de sa demande.

Voyons à quoi ressemblent les premiers textes des 100 premiers frames :

In [12]:
# On affiche les premières utterances des 100 premiers dialogues
for i in range(100):
    print(frames[i].get("turns", [])[0].get("text", ""))

I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.
Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.
Hello there i am looking to go on a vacation with my family to Gotham City, can you help me?
Hi I'd like to go to Caprica from Busan, between Sunday August 21, 2016 and Wednesday August 31, 2016
Hello, I am looking to book a trip for 2 adults and 6 children for $21,300 or less. We are departing from Kochi for Denver.
Hey, i Want to go to St. Louis on the 17th of August
I'm looking for a trip to Gotham City leaving from Kakariko Village on Saturday, August 13, 2016. 3 adults for no more than $2400 USD.
Hello, I would like to book a 2-week trip leaving from Melbourne on August 27. I would like to go to Mannheim.
Hello, I am planning to book a trip to pittsborgh
Hi, I need to go to Mos Eisley for a wedding, leaving on Saturday, August 13, 2016 and returning on Tuesday, August 16, 2016. Pr

Les premiers textes semblent effectivement exprimer le besoin principal de l'utilisateur. On retrouvera quelques exceptions comme des phrases de salutation du type "Heyo!", "Hi !", etc...

### Indicateurs statistiques

Commençons par mettre nos données dans un dataframe :

In [37]:
# On crée les lignes
rows = []
for i, frame in enumerate(frames):
    for turn in frame["turns"]:
        rows.append({
            "frame_id": i,
            "author": turn["author"],
            "text": turn["text"]
        })

# On crée le dataframe
frames_df = pd.DataFrame(rows)

# On ajoute le nombre de mots par texte
frames_df["word_nb"] = frames_df["text"].apply(lambda x: len(x.split()))

# On convertit la variable catégorique "author" en one hot encoding
frames_df = pd.get_dummies(frames_df, columns=["author"])

frames_df.shape

(19986, 5)

In [39]:
frames_df.head()

,frame_id,text,word_nb,author_user,author_wizard
0,0,I'd like to book a trip to Atlantis from Capri...,25,1,0
1,0,"Hi...I checked a few options for you, and unfo...",29,0,1
2,0,"Yes, how about going to Neverland from Caprica...",23,1,0
3,0,I checked the availability for this date and t...,21,0,1
4,0,I have no flexibility for dates... but I can l...,18,1,0


Voici des indicateurs statistiques du nombre de mots dans les textes :

In [41]:
frames_df[["word_nb"]].describe().T

,count,mean,std,min,25%,50%,75%,max
word_nb,19986.0,12.602172,10.892189,1.0,5.0,9.0,16.0,112.0


On constate que les textes ont en moyenne 13 mots et 75% des textes font moins de 16 mots. En effet, il s'agit d'un système de chat, ce qui explique que les textes sont plutôt courts.

Le texte le plus court compte 1 mot et le plus long en compte 112.

Voici des indicateurs statistiques du nombre de textes par frame et du type d'auteur :

In [43]:
frames_df.groupby("frame_id").agg(
    {
        "text": "count",
        "author_user": "sum",
        "author_wizard": "sum",
    }
).describe().T

,count,mean,std,min,25%,50%,75%,max
text,1369.0,14.598977,6.957727,1.0,10.0,14.0,19.0,48.0
author_user,1369.0,7.601899,3.521658,1.0,5.0,7.0,10.0,24.0
author_wizard,1369.0,6.997078,3.452898,0.0,5.0,7.0,9.0,24.0


On constate que l'on a en moyenne 15 textes par frames.

En moyenne, 8 textes sont de l'utilisateur et 7 textes du bot. Ceci semble cohérent puisque la tâche principale du bot est de récolter des informations afin de pouvoir trouver un vol/hôtel pour l'utilisateur. Il y a donc certainement beaucoup de questions/réponses, ce qui explique cet équilibre.

## Analyse des labels

Chaque turn a été labellisé. Voyons à quoi ressemblent les labels du 1er turn du 1er frame :

In [44]:
pprint_dict(frames[0]["turns"][0]["labels"]["acts_without_refs"])

[
  {
    "args": [
      {
        "val": "book",
        "key": "intent"
      }
    ],
    "name": "inform"
  },
  {
    "args": [
      {
        "val": "Atlantis",
        "key": "dst_city"
      },
      {
        "val": "Caprica",
        "key": "or_city"
      },
      {
        "val": "Saturday, August 13, 2016",
        "key": "str_date"
      },
      {
        "val": "8",
        "key": "n_adults"
      },
      {
        "val": "1700",
        "key": "budget"
      }
    ],
    "name": "inform"
  }
]


Le 1er élément est une liste d'intent qui doit indiquer l'intention principale du texte.

Le 2ème élément est une liste d'entities qui sont des informations détaillées de l'intent.

### Distribution des labels

Voyons la distribution des labels pour les textes de l'utilisateur :

In [116]:
label_values = defaultdict(list)
for frame in frames:
    for turn in frame["turns"]:
        # On ne s'intéresse qu'aux textes de l'utilisateur
        if turn["author"] != "user":
            continue

        for args in turn["labels"]["acts_without_refs"]:
            # On va ajouter chaque valeur dans la liste du label correspondant
            for label in args["args"]:
                try:
                    label_values[label["key"]].append(label.get("val", ""))
                except:
                    print(label)

In [83]:
# On crée un dataframe
labels = pd.Series(label_values).to_frame("all_values")

# On ajoute des variables concernant toutes les valeurs de chaque label
labels["all_values_nb"] = labels["all_values"].apply(len)
all_values_total = labels["all_values_nb"].sum()
labels["all_values_ratio"] = labels["all_values_nb"] / total

# On ajoute des variables concernant les valeurs uniques de chaque label
labels["unique_values"] = labels["all_values"].apply(set)
labels["unique_values_nb"] = labels["unique_values"].apply(len)

# On range les données par plus effectif
labels = labels.sort_values("all_values_nb", ascending=False)

labels.shape

(50, 5)

In [84]:
labels

,all_values,all_values_nb,all_values_ratio,unique_values,unique_values_nb
dst_city,"[Atlantis, Neverland, Mos Eisley, Neverland, M...",3541,0.202795,"{Manaus, KYOTO, St. Petersburg, Curritiba, Phi...",364
intent,"[book, book, book, book, book, book, book, boo...",2006,0.114885,{book},1
or_city,"[Caprica, Caprica, Atlantis, Caprica, Gotham C...",1962,0.112365,"{Manaus, neverland, St. Petersburg, Phili, Osa...",333
str_date,"[Saturday, August 13, 2016, August 13, 2016, -...",1544,0.088426,"{August 17, Sept 8, 28th, Friday September 2nd...",447
budget,"[1700, 1900, 2100, None, $2500, $2100, $2200, ...",1246,0.071359,"{$25600, 13700, 23300, 23 hundred dollars, 360...",395
n_adults,"[8, 5, 5, 2, 2, 3, with 14 adults, 1, 12, Just...",1029,0.058931,"{I'm with my wife, my 3 wives, my husband and ...",262
end_date,"[Wednesday August 31, 2016, Sunday August 28th...",1022,0.058530,"{mon, Aug 24, 2nd of Oct, Sept 8, 28th, August...",320
ref_anaphora,"[that's, this package, that, options, the hote...",874,0.050054,"{The hotel's, that place, the previous hotel, ...",149
category,"[None, 4-star, None, 3, None, None, None, None...",690,0.039517,"{even better, 2.5, flagship, the best, four, f...",85
price,"[None, None, None, None, None, None, too expen...",539,0.030869,"{expensive, 786.39, CHEAPER, cheaper, 4268, 28...",49


20% des valeurs lebellisées ont le label `dst_city` qui représente la ville de destination.

On constate aussi que les 5 labels qui nous intéressent sont parmis les 7 labels les plus représentés.

Enfin, le label `intent` ne possède qu'une seule valeur qui est `book` et qui représente donc l'intention de réserver un vol et/ou un hôtel.

## Transformation des données

Nous allons maintenant transformer nos données. Nous allons filtrer les données qui nous intéressent et nous allons les mettre au format LUIS. Ensuite, nous allons splitter nos données en un jeu d'entrainement et en un jeu de test. Pour finir, nous allons enregistrer nos données dans le Datastore et créer un Dataset.

Commençons par observer les paramètres du modèle actuel :

In [106]:
print("Intents :", [i["name"] for i in params["model"]["intents"]])
print("Entities :", [i["name"] for i in params["model"]["entities"]])

Intents : ['book_flight', 'None']
Entities : ['budget', 'from_city', 'from_dt', 'to_city', 'to_dt']


Nous allons faire correspondre un label à chaque entité de notre modèle :

In [100]:
# Mapping entre les labels du jeu de données et ceux de LUIS
label_to_entity = {
    "or_city": "from_city",
    "dst_city": "to_city",
    "str_date": "from_dt",
    "end_date": "to_dt",
    "budget": "budget"
}

On va convertir les turns utilisateur au format LUIS. Nous allons ajouter quelques variables pour nous aider à filtrer les données :

In [143]:
# On convertit les turns utilisateur du jeu de données pour LUIS
df = user_turns_to_luis_ds(
    frames,
    "book_flight",
    label_to_entity
)
    
df.shape

(10407, 11)

Comme nous l'avons vu au début de l'analyse des textes, le premier turn de chaque dialogue est celui qui contient le plus d'information. Nous n'allons donc conserver que ces textes là :

In [145]:
df = df[df["user_turn_id"] == 0]
df.shape

(1369, 11)

In [146]:
df.head()

,user_turn_id,text,intent,entities,entity_total_nb,from_city_nb,to_city_nb,from_dt_nb,to_dt_nb,budget_nb,text_word_nb
0,0,I'd like to book a trip to Atlantis from Capri...,book_flight,"[{'entity': 'to_city', 'startPos': 27, 'endPos...",4,1,1,1,0,1,25
4,0,"Hello, I am looking to book a vacation from Go...",book_flight,"[{'entity': 'to_city', 'startPos': 59, 'endPos...",3,1,1,0,0,1,16
14,0,Hello there i am looking to go on a vacation w...,book_flight,"[{'entity': 'to_city', 'startPos': 63, 'endPos...",1,0,1,0,0,0,20
25,0,"Hi I'd like to go to Caprica from Busan, betwe...",book_flight,"[{'entity': 'to_city', 'startPos': 21, 'endPos...",4,1,1,1,1,0,19
32,0,"Hello, I am looking to book a trip for 2 adult...",book_flight,"[{'entity': 'budget', 'startPos': 67, 'endPos'...",3,1,1,0,0,1,25


Voyons à quoi ressemblent les textes de l'intent `None` :

In [149]:
for text in df[df["intent"] == "None"]["text"].iloc[:30]:
    print(text)

Hello, I have 15 vacation days available between June 1st and August 31st. I am leaving from Theed. I would like to go somewhere with lots of sunshine.
Hi!
Hi! I'd like to go to Boston from Mos Eisley on August 15th.
Heyo!
Good morning.
Hi. I need to book a vacation to Long Beach between August 25 and September 3. Departure is from Paris
Hi we're from Miami and we want to go to paris, can you help out?
Hello wozbot!
Hi im fro termina and i want to go on vacation on August 13th
Hi i am looking to go to Punta Cana with my three friends
ay whats up?
Hi. I'd like to find a trip to San Diego. I can leave from Belo Horizonte. I'd like to travel between August 27th and August 31st if possible. My dates are flexible.
I'd like to book a trip from Chicago to San Diego between August 26th and September 5th.
hi there
me again... I'm still burnt out from work
hi my wife and i wanna bring our 7 little lads n lassies to kochi leaving from san diego  we can spend 28500 
hi i need to get to mexico plea

On s'apperçoit qu'il y a des doublons et que certaines phrases représentent une intention de réserver un vol/hôtel.

Nous allons donc commencer par supprimer les doublons dans les textes :

In [151]:
# On supprime les doublons
df = df.drop_duplicates(["text"])
df.shape

(1329, 11)

Voyons la distribution des intents :

In [152]:
df["intent"].value_counts()

book_flight    1135
None            194
Name: intent, dtype: int64

On va maintenant mondifier l'intent des textes labellisé à `None` et qui ont au moins une entité liée à la réservation d'un vol :

In [155]:
mask = (df["intent"] == "None") & (df["entity_total_nb"] > 0)
df.loc[mask, "intent"] = "book_flight"

df["intent"].value_counts()

book_flight    1260
None             69
Name: intent, dtype: int64

Observons quelques exemple des textes ayant l'intent `None` :

In [156]:
for text in df[df["intent"] == "None"]["text"].iloc[:30]:
    print(text)

Hi!
Heyo!
Good morning.
Hello wozbot!
ay whats up?
hi there
me again... I'm still burnt out from work
hey
hello hello
HEY
haayyyyy
Hi
Hello :slightly_smiling_face:
hello
helllo
hi
Hello,
Hi there
Hey
Heyooooo
suh
Hey there!
sup 
spring break gurl whats up yaassss
Yo dude.
Howdy! So how does this work? I'm not too good with technology, I'm more of a pen and paper kind of guy
Hey how are you?
Hi there,
hello.
hello................


Observons quelques indicateurs statistiques après nettoyage de notre jeu de données :

In [157]:
df[
    ["text_word_nb", "entity_total_nb"] + [f"{i}_nb" for i in label_to_entity.values()]
].describe().T

,count,mean,std,min,25%,50%,75%,max
text_word_nb,1329.0,17.469526,11.829137,1.0,9.0,15.0,23.0,74.0
entity_total_nb,1329.0,1.860798,1.272167,0.0,1.0,2.0,3.0,5.0
from_city_nb,1329.0,0.607223,0.490091,0.0,0.0,1.0,1.0,2.0
to_city_nb,1329.0,0.651618,0.476637,0.0,0.0,1.0,1.0,1.0
from_dt_nb,1329.0,0.278405,0.448382,0.0,0.0,0.0,1.0,1.0
to_dt_nb,1329.0,0.159518,0.366297,0.0,0.0,0.0,0.0,1.0
budget_nb,1329.0,0.164033,0.370445,0.0,0.0,0.0,0.0,1.0


Les 1er textes ont en moyenne 17 mots. Ils sont donc plus longs que les autres textes. En effet, on a pu voir que l'utilisateur va souvent exprimer sa demande ainsi que des informations complémentaires dans ce 1er texte.

L'utilisateur va en moyenne spécifier la ville de départ dans 60% des cas et la ville d'arrivée dans 65% des cas. On retrouve ensuite la date de départ dans 29% des cas et la date d'arrivée et le budget dans seulement 16% des cas. Notre bot devra donc être en mesure de questionner l'utilisateur pour récupérer les données manquantes.

La documentation de LUIS ([en savoir plus](https://docs.microsoft.com/en-us/azure/cognitive-services/luis/luis-concept-app-iteration)) nous conseille de créer un premier modèle avec environ 30 exemples par intent. Il s'agit surtout du cas où nous devons créer et labelliser les textes à la main. Etant donnée que nous avons un dataset déjà labellisé, nous pourrions utiliser tout notre jeu de données. Cependant, les bonnes pratiques de LUIS ([en savoir plus](https://docs.microsoft.com/en-us/azure/cognitive-services/luis/luis-concept-best-practices#do-balance-your-utterances-across-all-intents)) nous recommande aussi de créer des jeu de données équilibré en prenant le même nombre d'exemple par intent.

Commençons par voir combien de textes ont toutes les entitées :

In [172]:
book_flight_utterances = df[(df["intent"] == "book_flight") & (df["entity_total_nb"] >= len(label_to_entity))]
book_flight_utterances.shape

(33, 11)

Nous avons 33 individus. Nous allons ajouter 167 textes au hazard afin d'obtenir 200 individus. Nous aurons ainsi 2,8 fois plus d'individus que pour l'intent `None`. Notre jeu de données sera donc déséquilibré mais cela n'est pas grave car il sera beaucoup plus simple par la suite d'ajouter des individus pour l'intent `None`. En effet, ces dernier n'ont pas d'entités à labelliser.

In [173]:
# On échantillonne 167 individus au hazard
tmp = df[(df["intent"] == "book_flight") & (df["entity_total_nb"] < len(label_to_entity))]
tmp = tmp.sample(n=167, random_state=RANDOM_SEED)

# On concatène les données
book_flight_utterances = pd.concat([book_flight_utterances, tmp])
book_flight_utterances.shape

(200, 11)

On récupère ensuite les textes de type `None` :

In [176]:
none_utterances = df[df["intent"] == "None"]
none_utterances.shape

(69, 11)

On finit en concaténant nos données :

In [177]:
utterances = pd.concat([book_flight_utterances, none_utterances])
utterances.shape

(269, 11)

## Split des données

Nous allons séparer nos données en un jeu d'entrainement et un jeu de test.

In [210]:
# On prend 70% des données pour le jeu d'entrainement
utterances_train = utterances.sample(frac=0.7, random_state=RANDOM_SEED)

print(f"Le jeu d'entrainement compte {utterances_train.shape[0]} textes labellisés.")

Le jeu d'entrainement compte 188 textes labellisés.


In [211]:
# On crée le jeu de test avec les 30% restant
utterances_test_idx = utterances.index.difference(utterances_train.index)
utterances_test = utterances.loc[utterances_test_idx]

print(f"Le jeu de test compte {utterances_test.shape[0]} textes labellisés.")

Le jeu de test compte 81 textes labellisés.


On met les données au format LUIS :

In [184]:
utterances_train = utterances_train[["text", "intent", "entities"]].to_dict("records")

Voyons à quoi resemble le 1er élément de notre jeu d'entrainement :

In [185]:
pprint_dict(utterances_train[0])

{
  "text": "I would like to find a vacation between the 24th and 27th from San Francisco to Fort Lauderdale for 3800",
  "intent": "book_flight",
  "entities": [
    {
      "entity": "from_dt",
      "startPos": 44,
      "endPos": 47,
      "children": []
    },
    {
      "entity": "to_dt",
      "startPos": 53,
      "endPos": 56,
      "children": []
    },
    {
      "entity": "from_city",
      "startPos": 63,
      "endPos": 75,
      "children": []
    },
    {
      "entity": "to_city",
      "startPos": 80,
      "endPos": 94,
      "children": []
    },
    {
      "entity": "budget",
      "startPos": 100,
      "endPos": 103,
      "children": []
    }
  ]
}


Pour le jeu de test, nous devons respecter le format spécifié dans la documentation de LUIS ([en savoir plus](https://docs.microsoft.com/en-us/azure/cognitive-services/luis/luis-how-to-batch-test#batch-syntax-template-for-intents-with-entities)) :

In [187]:
utterances_test = utterances_test[["text", "intent", "entities"]].to_dict("records")
utterances_test = {
    "LabeledTestSetUtterances": utterances_test
}

Voyons à quoi resemble le 1er élément de notre jeu de test :

In [188]:
pprint_dict(utterances_test["LabeledTestSetUtterances"][0])

{
  "text": "The police are after me, and I need to leave Kobe between August 27 and September 2.",
  "intent": "book_flight",
  "entities": [
    {
      "entity": "from_city",
      "startPos": 45,
      "endPos": 48,
      "children": []
    },
    {
      "entity": "from_dt",
      "startPos": 58,
      "endPos": 66,
      "children": []
    },
    {
      "entity": "to_dt",
      "startPos": 72,
      "endPos": 82,
      "children": []
    }
  ]
}


## Enregistrement des datasets

Nous allons enregistrer nos données au format JSON dans le Datastore :

In [192]:
with tempfile.TemporaryDirectory() as tmp_dir_name:
    # On enregistre les données
    file_path = os.path.join(tmp_dir_name, "utterances_train.json")
    with open(file_path, "w") as f:
        json.dump(utterances_train, f)
        
    file_path = os.path.join(tmp_dir_name, "utterances_test.json")
    with open(file_path, "w") as f:
        json.dump(utterances_test, f)
    
    # On upload tous les fichiers dans le datastore
    ds = Dataset.File.upload_directory(
        tmp_dir_name,
        target=(datastore, "utterances/" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")),
        overwrite=True,
        show_progress=True
    )

Validating arguments.
Arguments validated.
Uploading file to utterances/2022_01_02


Unrecognized authentication type: None


Uploading an estimated of 2 files
Uploading /tmp/tmpr4lpf75r/utterances_test.json
Uploaded /tmp/tmpr4lpf75r/utterances_test.json, 1 files out of an estimated total of 2
Uploading /tmp/tmpr4lpf75r/utterances_train.json
Uploaded /tmp/tmpr4lpf75r/utterances_train.json, 2 files out of an estimated total of 2
Uploaded 2 files
Creating new dataset


On crée ensuite un Dataset Azure à partir de ces fichiers :

In [197]:
ds = ds.register(
    workspace=ws,
    name="utterances",
    description="Train and test utterances",
    create_new_version=True
)

In [198]:
print(f"Création du dataset '{ds.name}' version {ds.version}.")

Création du dataset 'utterances' version 1.


# Enregistrement des paramètres de LUIS sur Github

Il existe plusieurs façons de créer un modèle LUIS. Nous avons d'abord testé le tutorial suivant qui utilise le SDK Python : [quickstart](https://docs.microsoft.com/en-us/azure/cognitive-services/luis/client-libraries-rest-api?tabs=windows&pivots=programming-language-python).

Nous nous sommes ensuite apperçut que l'on pouvait importer et exporter notre modèle au format JSON ([en savoir plus](https://docs.microsoft.com/en-us/azure/cognitive-services/luis/app-schema-definition)). Ce format s'avère pratique pour versionner le modèle et il reste assez simple pour être manipulé dans le cadre de recherche d'hyperparamètres. Cette méthode utilise le SDK python ainsi que l'API REST de LUIS. Les paramètres du modèle LUIS ainsi que les informations du jeu de données sont stockés dans le fichier `../P10_02_luis/params.json`. C'est ce fichier qui sera ensuite utilisé par les Github actions pour déployer le modèle en production.

## Mise à jour des paramètres de LUIS

On va donc commencer par mettre à jour le fichier JSON, notamment les informations du jeu de données précédemment créé.

In [202]:
# On copie les paramètres du modèle actuel
new_params = params.copy()

In [203]:
# On met à jour les paramètres avec le jeu de données que l'on a créé
new_params["dataset"]["name"] = ds.name
new_params["dataset"]["version"] = ds.version

In [204]:
# On enregistre les nouveaux paramètres
with open("../P10_02_luis/params.json", "w") as f:
    json.dump(new_params, f)

## Evaluation du nouveau modèle

Nous allons utiliser les mêmes briques logicielles que celles utilisées pour le déployement du modèle en production. Elles sont disponible dans le fichier `../P10_02_luis/utils.py`.

In [205]:
# On commence par créer un nom de version temporaire
tmp_app_version = "tmp"

In [206]:
# On crée ensuite une nouvelle version du modèle.
# Un premier modèle a déjà été créé dans le script de création
# des ressources de LUIS "../P10_02_luis/luis_create.sh".
# Ce 1er modèle a été créé à partir du fichier "../P10_02_luis/params.json".
create_new_version(env, tmp_app_version, new_params["model"], utterances_train)

In [207]:
# On entraine notre modèle
train(env, tmp_app_version)

In [208]:
# On le déploie sur l'environnement de test
deploy(env, tmp_app_version, "staging")

In [209]:
# On évalue le modèle avec le jeu de test
res = evaluate(env, is_staging=True, utterances=utterances_test)
res

,model_name,model_type,precision,recall,f_score
0,book_flight,Intent Classifier,0.92,1.00,0.96
1,None,Intent Classifier,1.00,0.78,0.88
2,from_dt,Entity Extractor,0.86,0.90,0.88
3,to_dt,Entity Extractor,0.88,1.00,0.93
4,budget,Entity Extractor,0.88,1.00,0.93
5,from_city,Entity Extractor,0.57,0.85,0.69
6,to_city,Entity Extractor,0.50,1.00,0.67


Pour chaque intent et chaque entity, on obtient 3 scores :
- `precision` : parmis les labels prédit sur chaque mot, indique lesquels sont corrects.
- `recall` : parmis les labels à détecter, indique lesquels ont été détéctés par le modèle.
- `f_score` : moyenne harmonique de la precision et du recall.

Pour un premier modèle, on obtient des scores satisfaisants.

On remarque que le recall de l'intent `None` est à 0.78. Le modèle semble donc louper certain de ces intents.

On s'apperçoit aussi que la precision des entités `from_city` et `to_city` sont à 0.57 et 0.5. Le modèle semble donc créer beaucoup de faux positifs sur ces entités.

In [ ]:
# On supprime la version temporaire
delete(env, tmp_app_version)

## Enregistrement du Github

Commiter et pusher sur Github les modifications du fichier `../P10_02_luis/params.json` afin qu'il puisse être utilisé lors du déploiement du modèle.